In [1]:
import xarray as xr
import numpy as np
import netCDF4
import pandas as pd
from scipy import signal

In [2]:
var = "psl"
arr = []
for ens_index in range(1,17):
        filename = "/work/uo1075/u241321/data/u241321/data_cdo/psl_DJF_assi/"+var+"_Amon_MPI-ESM-LR_asSEIKERAf_r"+str(ens_index)+"i8p4_196901-201912_ym_g1d0.nc"
        arr.append(xr.load_dataset(filename,decode_times=False).assign_coords(ensemble=ens_index))

In [3]:
data = xr.concat(arr, dim="member")

In [4]:
annual_pre = data ['psl']

In [6]:
# 1969.12. 1970.1.2 = 1970 
# 1970.12.  1971.1.2 = 1971
# 2018.12. 2019.1.2. = 2019
# 1970-2019
annual_prediction = annual_pre.transpose("time", "member", "lat", "lon") # 
dat = np.mean(annual_prediction,axis=0)
dif = annual_prediction-dat

In [7]:
dropped = annual_prediction.stack(feature=("member","lat","lon")).dropna(dim="feature")
detrend = signal.detrend(dropped ,axis=0)
feature = dropped .coords["feature"]
time = dropped .coords["time"]
detrend = xr.DataArray(detrend, dims = ["time","feature"], coords = {"time":time,"feature":feature}).unstack()


In [8]:
detrend.to_netcdf("/work/uo1075/u241321/data/psl_DJF_1970-2019_assi_dt.nc")

In [9]:
dif.to_netcdf("/work/uo1075/u241321/data/psl_DJF_1970-2019_assi.nc")  # 